In [ ]:
from huggingface_hub import login

# Insira seu token de acesso pessoal
login("Your_HuggingFace_API_Key",add_to_git_credential=True)


In [ ]:
import torch
from transformers import pipeline

model_id = "meta-llama/Llama-3.2-3B-Instruct"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.float16,
    device = 'cuda',
    
)



In [ ]:
import pandas as pd

# Função para carregar e formatar o FAQ a partir de um CSV
def carregar_faq_do_csv(caminho_csv):
    try:
        # Lendo o CSV com pandas
        df = pd.read_csv(caminho_csv)
        # Verifica se as colunas "Pergunta" e "Resposta" existem
        if "Pergunta" not in df.columns or "Resposta" not in df.columns:
            raise ValueError("O arquivo CSV deve conter colunas 'Pergunta' e 'Resposta'.")
        
        # Concatena as perguntas e respostas no formato desejado
        faq_text = "\n".join(f"- {row['Pergunta']} {row['Resposta']}" for _, row in df.iterrows())
        return faq_text
    except Exception as e:
        print(f"Erro ao carregar o FAQ: {e}")
        return ""

# Carregar o CSV
caminho_csv = "FAQ.csv"  # Substitua pelo caminho do seu arquivo CSV
FAQ = carregar_faq_do_csv(caminho_csv)

In [ ]:
import tkinter as tk
from tkinter import ttk

# Função para gerar a resposta
def gerar_resposta(pergunta, faq, fallback="Não sei responder a essa pergunta. Por favor, tente perguntar de outra forma ou consulte nosso FAQ."):
    prompt = f"""
    Você é um assistente que responde perguntas com base exclusivamente no seguinte FAQ. 
    Não inclua o texto do FAQ na resposta, apenas use-o como referência para responder à pergunta.

    FAQ:
    \"\"\"{faq}\"\"\"

    Responda à seguinte pergunta de forma direta e objetiva:

    Pergunta: {pergunta}
    Resposta:
    """
    # Gera a resposta com o modelo
    outputs = pipe(
        prompt,
        max_new_tokens=100,  # Limite o número de tokens gerados para respostas curtas
        pad_token_id=50256  # Configuração explícita para evitar warnings
    )
    # Extrai somente a parte gerada como resposta
    resposta_gerada = outputs[0]["generated_text"].strip()
    
    # Remove o prompt inicial da resposta, caso seja incluído pelo modelo
    resposta = resposta_gerada.split("Resposta:", 1)[-1].strip()

    # Remove repetições redundantes
    palavras = resposta.split()
    resposta_filtrada = " ".join(dict.fromkeys(palavras))  # Remove palavras repetidas consecutivas
    
    # Verifica se a resposta é válida, caso contrário, usa o fallback
    if not resposta_filtrada or "não tenho informações" in resposta_filtrada.lower():
        return fallback
    
    return resposta_filtrada

# Função para processar a pergunta do usuário
def processar_pergunta():
    pergunta = entrada_pergunta.get()  # Obtem a pergunta do campo de entrada
    resposta = gerar_resposta(pergunta, FAQ)  # Gera a resposta
    label_resposta.config(text=f"Resposta: {resposta}")  # Exibe a resposta


# Configuração da janela principal
root = tk.Tk()
root.title("Assistente FAQ")
root.geometry("600x400")
root.configure(bg="#0f0fff")  # Cor de fundo azul claro


# Campo de entrada de pergunta
label_pergunta = tk.Label(root, text="Digite sua pergunta:")
label_pergunta.pack(pady=(20, 5))  # Espaçamento superior maior
label_pergunta.config(bg="#f0f8ff", font=("Arial", 14))

entrada_pergunta = tk.Entry(root, width=50)
entrada_pergunta.pack(pady=10, padx=20)  # Espaçamento interno horizontal
entrada_pergunta.config(font=("Arial", 12))

# Botão para enviar a pergunta
botao_enviar = ttk.Button(root, text="Obter Resposta", command=processar_pergunta)
botao_enviar.pack(pady=10)
botao_enviar.config(style="TButton")

# Área para exibir a resposta
label_resposta = tk.Label(root, text="Resposta: ")
label_resposta.pack(pady=20, padx=20)  # Adicionar espaçamento lateral para respostas longas
label_resposta.config(bg="#f0f8ff", font=("Arial", 12), wraplength=500, fg="#333")


# Executa o loop principal da interface
root.mainloop()